In [1]:
import numpy as np
import matplotlib.pyplot as plot
import pandas as pd


In [2]:
stations=pd.read_csv("station.csv")
trips = pd.read_csv("trip.csv")
trips=trips=trips.loc[trips.start_date.str.contains('8/'+'\S'+'/2014')]
trips=trips.loc[:,('start_station_name','start_station_id','end_station_name','end_station_id')]
stations=stations.loc[:]

In [3]:
import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')
    
import pyspark_csv as pycsv
sc.addPyFile('pyspark_csv.py')

ImportError: No module named pyspark

In [4]:
import pyspark.sql as pysql

sqlCtx = pysql.SQLContext(sc)

ImportError: No module named pyspark.sql

In [5]:
trip_rdd=sc.textFile('trip.csv')
trips_df =pycsv.csvToDataFrame(sqlCtx,trip_rdd)

NameError: name 'sc' is not defined

In [6]:
#el rdd de los trips
data=trips_df.rdd

NameError: name 'trips_df' is not defined

In [7]:
#de la forma ( (start_station_id,end_station_id) , 1)  
frecuency=data.map(lambda x: ((x.start_staton_id,x.end_station_id),1) ).reduceByKey(lambda x,y:x+y).collect()

NameError: name 'data' is not defined

In [8]:
trips_df.registerTempTable('trips')

NameError: name 'trips_df' is not defined

In [9]:
max_index=sqlCtx.sql('select max(start_station_id) from trips').collect()
matrix=[ np.zeros(max_index+1) for i in range(max_index+1)]


for (row,col),val in frecuency:
    matrix[row][col]=val

NameError: name 'sqlCtx' is not defined

In [10]:

%matplotlib inline

plot.matshow(matrix)
plot.show()


NameError: name 'matrix' is not defined